In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 1. Implement SoftMax Function from Scratch & Compare with PyTorch
def softmax_numpy(x):
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / np.sum(exp_x, axis=0)

# Test input
input_data = np.array([2.0, 1.0, 0.1])

# SoftMax using NumPy
numpy_result = softmax_numpy(input_data)

# SoftMax using PyTorch
torch_result = torch.nn.functional.softmax(torch.tensor(input_data), dim=0).numpy()

# Compare results
print("NumPy SoftMax:", numpy_result)
print("PyTorch SoftMax:", torch_result)
print("Are they equal?", np.allclose(numpy_result, torch_result))

In [7]:
# 2. Understanding Cross Entropy Loss
def binary_cross_entropy(y_pred, y_true, epsilon=1e-12):
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)  # Avoid log(0)
    return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

cases = [(0.1, 0), (0.9, 1), (0.2, 1), (0.8, 0)]

for pred, gt in cases:
    loss = binary_cross_entropy(pred, gt)
    print(f"Pred: {pred}, GT: {gt}, Loss: {loss}")

Pred: 0.1, GT: 0, Loss: 0.10536051565782628
Pred: 0.9, GT: 1, Loss: 0.10536051565782628
Pred: 0.2, GT: 1, Loss: 1.6094379124341003
Pred: 0.8, GT: 0, Loss: 1.6094379124341005


In [8]:
# 3. Understanding Adam Optimizer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize model
model = SimpleNN()

# List of different learning rates
learning_rates = [0.1, 0.01, 0.001, 0.0001]
optimizers = {lr: optim.Adam(model.parameters(), lr=lr) for lr in learning_rates}

for lr in learning_rates:
    print(f"Created Adam optimizer with learning rate: {lr}")

Created Adam optimizer with learning rate: 0.1
Created Adam optimizer with learning rate: 0.01
Created Adam optimizer with learning rate: 0.001
Created Adam optimizer with learning rate: 0.0001


In [ ]:
# 4. Training a Fashion MNIST Classifier
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load Fashion MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, loss, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

In [ ]:
# 5. Evaluating the Model on Test Data
model.eval()
test_loss, correct, total = 0, 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")